In [0]:
import re
import sys

import math
import numpy as np
from sklearn.metrics import mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

In [2]:
#Installing dependencies for RDKit
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

--2020-05-30 08:38:58--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2020-05-30 08:38:58--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  81.12M   224MB/s    in 0.4s    

2020-05-30 08:38:58 (224 MB/s) - ‘Miniconda3-latest-Linux

In [0]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [4]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
root = "/content/drive/My Drive/"

In [0]:
import csv
def get_data(split):
    data_path =root +"QM9_Smiles_.csv"
    with open(data_path) as f:
        data = csv.reader(f)
    
        # Skip header
        next(data)
        
        # Get smiles and targets
        smiles, Y = [], []
        count=0
        for row in data:
            if (count<16000):
              smiles.append(row[0])
              Y.append(row[1:13])
              count=count+1
    
    return smiles, Y

AllSmiles, All = get_data('train')


In [0]:
import pandas as pd
import numpy as np

In [8]:
dtype = [('Col1','int32'), ('Col2','float32'), ('Col3','float32'),('Col4','int32'), ('Col5','float32'), ('Col6','float32'),('Col7','int32'), ('Col8','float32'), ('Col9','float32'),('Col10','int32'), ('Col11','float32'), ('Col12','float32'),('Col13','float32'), ('Col14','float32'),('Col15','float32'), ('Col16','float32'),('Col17','float32')]
values = All
index = ['Row'+str(i) for i in range(1, len(values)+1)]

df = pd.DataFrame(values, index=index).astype(float)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,5.0509,78.30,-0.2676,0.0141,0.2818,1120.8417,0.137571,-380.797809,-380.790510,-380.789566,-380.829714,28.666
Row2,2.5256,55.40,-0.2868,-0.0597,0.2271,1180.5099,0.064339,-525.783370,-525.775913,-525.774969,-525.816245,25.955
Row3,1.5279,83.85,-0.2229,0.0807,0.3036,1029.1179,0.196525,-367.186557,-367.178749,-367.177805,-367.218256,32.345
Row4,2.1100,72.11,-0.2575,-0.0123,0.2451,1230.8475,0.136089,-455.191771,-455.182799,-455.181854,-455.226039,32.000
Row5,2.7860,75.68,-0.2791,0.0257,0.3048,1496.1036,0.168867,-440.291337,-440.280566,-440.279622,-440.327569,37.969


In [9]:
#Double check we have the right values 
s= pd.DataFrame(AllSmiles, index=index)

s.head()

,0
Row1,N#CC1NC11C2CC1C2
Row2,OC1=NON=C1OC=O
Row3,CC12CCC1CN1CC21
Row4,CN=C1OC(=O)CC1N
Row5,CC(CO)C(CO)C#N


In [10]:
len(df)

16000

In [0]:
std=df[7].std()

In [12]:
std

40.180913619091854

In [13]:
# Normalize target values to have a mean of 0 and std of 1
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
data_scaled=scaler.fit_transform(df)
df = pd.DataFrame(data_scaled , index=index)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
Row1,1.515532,0.376722,-1.263346,0.065564,0.650691,-0.239353,-0.329338,0.758062,0.758037,0.758037,0.758089,-0.713853
Row2,-0.113146,-2.404221,-2.141010,-1.515652,-0.505019,-0.023702,-2.539103,-2.850370,-2.850411,-2.850411,-2.850322,-1.381283
Row3,-0.756606,1.050706,0.779965,1.492516,1.111284,-0.570858,1.449591,1.096821,1.096811,1.096811,1.096849,0.191892
Row4,-0.381184,-0.374982,-0.801658,-0.500074,-0.124713,0.158226,-0.374057,-1.093471,-1.093465,-1.093465,-1.093480,0.106955
Row5,0.054798,0.058553,-1.789030,0.314102,1.136638,1.116906,0.615015,-0.722626,-0.722573,-0.722573,-0.722688,1.576482


In [0]:
def get_data(smiles_array,target_array,start, end):
   # Get smiles and targets
    smiles, Y = [], []
    for i in range(start,end):
      smiles.append(smiles_array[i])
      Y.append(target_array[i])
    
    return smiles, Y

trainSmiles, trainY = get_data(AllSmiles, df[7],0,16000)
allSmiles = trainSmiles 

In [15]:
allSmiles[0]

'N#CC1NC11C2CC1C2'

In [0]:
class QM9Dataset(Dataset):
    def __init__(self, X, Y):
      self.X, self.Y = X, Y
      assert len(X) == len(Y)

    def __len__(self):
       return len(self.X)

    def __getitem__(self, i):
      return np.array(self.X[i]), self.Y[i]

In [0]:
batch_size = 64
lr = 1e-3
weight_decay = 1e-4
max_len = 25
embedding_size = 1
hidden_size = 64
output_size = 1  
dropout = 0.6
use_cuda = True

In [0]:
def param_count(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)
  
def mae(targets, preds):
    return mean_absolute_error(targets, preds)

In [19]:
# Define vocab
vocab = {char for smiles in AllSmiles for char in smiles}

print(f'Vocab = {vocab}')

# Create word to index mapping
padding_idx = 0
char_to_index = {char: index + 1 for index, char in enumerate(vocab)}
vocab_size = len(char_to_index) + 1

print(f'Vocab size = {vocab_size:,}')

Vocab = {')', '[', 'N', '2', '=', '#', '4', 'O', '3', ']', '5', 'F', '(', '1', 'C', 'H', '-', '+'}
Vocab size = 19


In [20]:
char_to_index 

{'#': 6,
 '(': 13,
 ')': 1,
 '+': 18,
 '-': 17,
 '1': 14,
 '2': 4,
 '3': 9,
 '4': 7,
 '5': 11,
 '=': 5,
 'C': 15,
 'F': 12,
 'H': 16,
 'N': 3,
 'O': 8,
 '[': 2,
 ']': 10}

In [21]:
biggest_mol_size = max([len(smiles) for smiles in allSmiles])
print("Total number of smiles= ", len(AllSmiles))
print("size of largest molecule = ", biggest_mol_size)

Total number of smiles=  16000
size of largest molecule =  27


In [22]:
trainX = [[char_to_index[char] for char in smiles] for smiles in trainSmiles]
print(f'Smiles string = {trainSmiles[0]}')
print(f'Indices of first train SMILES = {trainX[0]}')
print(f'Last five indices = {trainX[0][-5:]}')


print(f'Smiles string = {trainSmiles[2]}')
print(f'Indices of second train SMILES = {trainX[2]}')
print(f'Last five indices = {trainX[2][-5:]}')

Smiles string = N#CC1NC11C2CC1C2
Indices of first train SMILES = [3, 6, 15, 15, 14, 3, 15, 14, 14, 15, 4, 15, 15, 14, 15, 4]
Last five indices = [15, 15, 14, 15, 4]
Smiles string = CC12CCC1CN1CC21
Indices of second train SMILES = [15, 15, 14, 4, 15, 15, 15, 14, 15, 3, 14, 15, 15, 4, 14]
Last five indices = [14, 15, 15, 4, 14]


In [23]:
trainX = [seq[:max_len] + [padding_idx] * (max_len - len(seq)) for seq in trainX]

print(f'Smiles string = {trainSmiles[0]}')
print(f'Indices of first train SMILES = {trainX[0]}')
print(f'Last five indices = {trainX[0][-5:]}')

print(f'Smiles string = {trainSmiles[2]}')
print(f'Indices of second train SMILES = {trainX[2]}')
print(f'Last five indices = {trainX[2][-5:]}')


Smiles string = N#CC1NC11C2CC1C2
Indices of first train SMILES = [3, 6, 15, 15, 14, 3, 15, 14, 14, 15, 4, 15, 15, 14, 15, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Last five indices = [0, 0, 0, 0, 0]
Smiles string = CC12CCC1CN1CC21
Indices of second train SMILES = [15, 15, 14, 4, 15, 15, 15, 14, 15, 3, 14, 15, 15, 4, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Last five indices = [0, 0, 0, 0, 0]


In [0]:
# Build Dataset
train = QM9Dataset(trainX, trainY)

# Build DataLoader
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

In [0]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, padding_idx, embedding_size, hidden_size, output_size, dropout):
        super(LSTMModel, self).__init__()
        
        # Embedding layer
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx=padding_idx)
        
        # LSTM (RNN)
        self.rnn_encoder = nn.LSTM(
            input_size=embedding_size,
            hidden_size=hidden_size,
            batch_first=True
        )
        
       # LSTM (RNN)
        self.rnn_decoder = nn.LSTM(
            input_size=hidden_size,
            hidden_size=embedding_size,
            batch_first=True
        )

        
        # Fully connected layer
        self.output = nn.Linear(hidden_size, output_size)
        
        # Dropout (regularization)
      #  self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):  # batch_size x seq_length
        # Embed
        embedded = self.embed(x)  # batch_size x seq_length x embedding_size
         
        # Run RNN
        o, _ = self.rnn_encoder(embedded)  # batch_size x seq_length x hidden_size
        m, _ = self.rnn_decoder(o)  # batch_size x seq_length x hidden_size
          
        # Dropout
       # o = self.dropout(o)  # batch_size x seq_length x hidden_size
        
        # Max pooling across sequence
        n, _ = torch.max(o, dim=1)    # batch_size x hidden_size
        
        # Output layer
        out = self.output(n)  # batch_size x output_size
        
        return embedded, m,n

In [0]:
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold
from sklearn.kernel_ridge import KernelRidge
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

"""Evaluates latent space quality via a linear regression downstream task."""
def linear_reg(X,Y,std):

  X_train, X_test_validate, y_train, y_test_validate = train_test_split(X, Y, test_size=0.4, random_state=101)
  X_valid, X_test, y_valid, y_test = train_test_split(X_test_validate, y_test_validate, test_size=0.5, random_state=101)
  params = { 'kernel':['laplacian'],'alpha':[0.001, 0.01,0.1,1,10,100]}
  #clf = GridSearchCV(KernelRidge(), params, cv=5,verbose=5)
  clf = KernelRidge(alpha=0.001, kernel='laplacian',)
  clf.fit(X_train, y_train)
  y_pred=clf.predict(X_test)
  y_predv=clf.predict(X_valid)
  mae_test = mean_absolute_error(y_test,y_pred)*std 
  mae_val = mean_absolute_error(y_valid,y_predv)*std 
  return mae_val,mae_test


def evaluate_embedding(embeddings, labels, std):
    xa, ya= embeddings.cuda().cpu(),labels.cuda().cpu(),
    x,y=np.array(xa.detach().numpy()),np.array(ya.detach().numpy())
    #print(x.shape, y.shape)
    val_accuracies,test_accuracies=linear_reg(x, y, std)

    return np.mean(val_accuracies),np.mean(test_accuracies)

In [0]:
def train(model, train_loader, optimizer, epoch, std):
    model.train()  # Set the nn.Module to train mode. 
    total_loss = 0
    total_mae = 0
    num_samples = len(train_loader.dataset)
    num_batches = 0
    val=[]
    test=[]
    for batch_idx, (data, target) in enumerate(train_loader):  # 1) get batch
        # Adjust dimensions of target and cast to float
        target = target.unsqueeze(1).float()
      
        # Move to cuda
        if next(model.parameters()).is_cuda:
            data, target = data.cuda(), target.cuda()
      
        # Reset gradient data to 0
        optimizer.zero_grad()
        
        # Get prediction for batch
        embed,output,n = model(data)
        
        # 2) Compute loss
        loss = F.mse_loss(output, embed)
        
        # 3) Do backprop
        loss.backward()
        
        # 4) Update model
        optimizer.step()
        
        y_v,y_t=evaluate_embedding(n, target, std)
        # Do book-keeping to track rmse and avg loss
        total_loss += loss.detach()  # Don't keep computation graph 
        num_batches += 1
        val.append(y_v)
        test.append(y_t)
    y_v=np.mean(val)
    y_t=np.mean(test)     
    return (y_v,y_t)

In [28]:
model = LSTMModel(vocab_size, padding_idx, embedding_size, hidden_size, output_size, dropout)
print(model)
print(f'Number of parameters = {param_count(model):,}')
epochs=10
# Move to cuda
if use_cuda and torch.cuda.is_available():
    model = model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, epochs+1):
    mae_v,mae_t = train(model, train_loader, optimizer, epoch,std)
    print('Epoch: {}'.format(epoch))
    print('Kernel Ridge Regression validation MAE : {}'.format(mae_v))
    print('Kernel Ridge Regression testing MAE : {}'.format(mae_t))

   

LSTMModel(
  (embed): Embedding(19, 1, padding_idx=0)
  (rnn_encoder): LSTM(1, 64, batch_first=True)
  (rnn_decoder): LSTM(64, 1, batch_first=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
)
Number of parameters = 17,504
Epoch: 1
Kernel Ridge Regression validation MAE : 25.831427926943324
Kernel Ridge Regression testing MAE : 25.70328891074025
Epoch: 2
Kernel Ridge Regression validation MAE : 24.605646489638428
Kernel Ridge Regression testing MAE : 24.42634088106529
Epoch: 3
Kernel Ridge Regression validation MAE : 24.911086704112027
Kernel Ridge Regression testing MAE : 24.835156506641066
Epoch: 4
Kernel Ridge Regression validation MAE : 24.73937784027495
Kernel Ridge Regression testing MAE : 24.693238637412062
Epoch: 5
Kernel Ridge Regression validation MAE : 24.873354920065932
Kernel Ridge Regression testing MAE : 24.047837603767743
Epoch: 6
Kernel Ridge Regression validation MAE : 24.543006207396793
Kernel Ridge Regression testing MAE : 24.461741204498825
Epoch